In [ ]:
versioninfo()

In [ ]:
g=6.0
ENV["JULIA_CUDA_MEMORY_LIMIT"] = convert(Int, round(g * 1024 * 1024 * 1024))

In [ ]:
ENV["JULIA_DEBUG"]="all"

# New version of main

- [X] refactor the code of main1
- [X] use seeding
- [X] pre-gen graphs and splitting
- [X] use cloud GPU for training
- [X] I don't need correlation experiments anymore

In [1]:
1+2

3

In [5]:
include("main.jl")

main (generic function with 1 method)

In [ ]:
main_ngraph()

In [ ]:
main_ersf124()

In [ ]:
main_ch3()

In [6]:
function main_cnn()
    for d in [10,20,50],
        (prefix, model_fn,nsteps) in [
#             ("CNN", ()->cnn_model(2), 3e4),
            ("CNN2-$(now())", ()->cnn_model(2, 128, (5,5), (2,2)), 3e4),
#             ("CNN2", ()->cnn_model(2, 128, (3,3), (1,1)), 3e4)
        ]
        
        specs = []
        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                    noise=:Gaussian, mat=:CH3,
                    ng=10000, N=1))
        end
        specs = Array{DataSpec}(specs)
        
        # print more frequently for CNN and FC to get more data to print
        test_throttle = if prefix == "EQ2" 10 else 1 end

        @info "training .." prefix d
        expID = exp_train(specs, model_fn,
                          # TODO I'll need to increase the training steps here
                          # CAUTION feed in the gtype in the model prefix
                          prefix="$prefix-ERSF-k1-d=$d", train_steps=nsteps,
                          test_throttle = test_throttle,
                          merge=true)
    end
end

main_cnn (generic function with 1 method)

In [ ]:
main_cnn()

┌ Info: training ..
│   prefix = CNN2-2021-01-20T15:40:39.803
│   d = 10
└ @ Main In[6]:21


(xdim, ydim) = (4, 3)
(xdim, ydim) = (4, 3)
ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 250
  x data shape: (10, 10, 2, 16000)
  y data shape: (10, 10, 16000)
  current index: 1]
test_ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 62
  x data shape: (10, 10, 2, 4002)
  y data shape: (10, 10, 4002)
  current index: 1]
size(x) = (10, 10, 2, 64)
size(y) = (10, 10, 64)


┌ Info: warming up model with x ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:143
┌ Info: warming up gradient ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:145


size(x) = (10, 10, 2, 64)


┌ Info: actual training ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:148
┌ Info: training ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:166
┌ Info: training for 30000 steps ..
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:222
┌ Warning: calls to Base intrinsics might be GPU incompatible
│   exception = (GPUCompiler.MethodSubstitutionWarning(log1p(x::Float32) in Base.Math at special/log.jl:360, log1p(x::Float32) in CUDA at /home/hebi/.julia/packages/CUDA/YeS8q/src/device/intrinsics/math.jl:85), Base.StackTraces.StackFrame[log1p at log.jl:360, broadcast_kernel at broadcast.jl:59])
└ @ GPUCompiler /home/hebi/.julia/packages/GPUCompiler/uTpNx/src/irgen.jl:68
┌ Warning: calls to Base intrinsics might be GPU incompatible
│   exception = (GPUCompiler.MethodSubstitutionWarning(log1p(x::Float32) in Base.Math at special/log.jl:360, log1p(x::Float32) in CUDA at /home/hebi/.julia/packages/CUDA/YeS8q/src/device/intrinsics/math.jl:85), Base.StackTraces.StackFrame[log1p at log.jl:360, broadcast_kernel a

(xdim, ydim) = (4, 3)
(xdim, ydim) = (4, 3)
ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 250
  x data shape: (20, 20, 2, 16000)
  y data shape: (20, 20, 16000)
  current index: 1]
test_ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 62
  x data shape: (20, 20, 2, 4002)
  y data shape: (20, 20, 4002)
  current index: 1]
size(x) = (20, 20, 2, 64)
size(y) = (20, 20, 64)


┌ Info: warming up model with x ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:143
┌ Info: warming up gradient ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:145


size(x) = (20, 20, 2, 64)


┌ Info: actual training ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:148
┌ Info: training ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:166
┌ Info: training for 30000 steps ..
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:222
Training... 48%|███████████████████▏                    |  ETA: 0:30:08m

In [ ]:
main_mat()

In [ ]:
main_ensemble_d()

# DEBUG

In [ ]:
main()

Ensemble

- [ ] I actually probably want to merge the datasets with the same d.
  This is because, I can merge the data to be more evenly distributed to fit the model.
- [ ] I probably want to adjust batch size to speed it up

In [ ]:
model = eq_model_fn() |> gpu

In [ ]:
ds, test_ds = spec2ds(DataSpec(d=20, k=1, gtype=:ER,
                    noise=:Gaussian, mat=:COV))

In [ ]:
x, y = next_batch!(test_ds) |> gpu

@show size(x)
@show size(y)

In [ ]:
model(x)

# model params

In [4]:
include("main.jl")

main (generic function with 1 method)

In [2]:
import Printf

In [ ]:
function test_size()
    # model size
    @info "FC model"
    for d in [10,20,50,100]
        Printf.@printf "%.2f\n" param_count(fc_model_fn(d)) / 1e6
    end
    @info "FC deep model"
    for d in [7, 10,15,20,25,30]
        Printf.@printf "%.2f\n" param_count(deep_fc_model_fn(d)) / 1e6
    end
    # EQ models is independent of input size
    @info "EQ model"
    Printf.@printf "%.2f\n" param_count(eq_model_fn(10)) / 1e6
    Printf.@printf "%.2f\n" param_count(deep_eq_model_fn(10)) / 1e6
end

In [ ]:
            ("EQ2", eq2_model_fn, 3e4),
            ("FC", ()->fc_model(d=d, ch=2, z=1024, nlayer=6), 3e4),
            ("FCreg", ()->fc_model(d=d, ch=2, z=1024, nlayer=6, reg=true), 3e4),
            ("CNN", ()->cnn_model(2), 3e4)

In [6]:
for d in [10,20,50,100]
    Printf.@printf "%.2f\n" param_count(fc_model(d=d, ch=2, z=1024, nlayer=6)) / 1e6
end

4.51
5.43
11.88
34.93


In [7]:
Printf.@printf "%.2f\n" param_count(eq2_model_fn()) / 1e6

1.45


In [14]:
Printf.@printf "%.2f\n" param_count(eq_model_fn()) / 1e6

0.36


In [8]:
Printf.@printf "%.2f\n" param_count(cnn_model(2)) / 1e6

0.04


cnn_model (generic function with 5 methods)

In [15]:
Printf.@printf "%.2f\n" param_count(cnn_model(1, 32)) / 1e6

0.04


In [11]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 32)) / 1e6

0.04


In [12]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 64)) / 1e6

0.15


In [13]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 128)) / 1e6

0.60


In [17]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 32, (5,5), (2,2))) / 1e6

0.11


In [18]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 64, (5,5), (2,2))) / 1e6

0.42


In [19]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 128, (5,5), (2,2))) / 1e6

1.65


# Test ERSF124 on individual types

In [11]:
function test_ersf124()
    for d in [10, 20],
        types in [(:ER, :SF), (:ER2, :SF2), (:ER4, :SF4)]
        
        ID = "EQ2-ERSF124-d=$d-ensemble"

        specs = []

        for gtype in types,
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                                  noise=:Gaussian, mat=:CH3))
        end
        specs = Array{DataSpec}(specs)

        exp_test(ID, specs, "TEST-types=$types")
    end
end

test_ersf124 (generic function with 1 method)

In [ ]:
test_ersf124()

# Test the ensemble model on unseen d

In [ ]:
# can I just run the test without saving? No because loading the main.jl is slow

In [1]:
include("main.jl")

main (generic function with 1 method)

In [21]:
function test_unseen_d()
    for ID in ["EQ2-CH3-d=[10,15,20]-ensemble","EQ2-CH3-d=[20,30,40]-ensemble"],
        d in [10, 15, 20, 30, 40, 50, 80, 100]

        specs = []

        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                                  noise=:Gaussian, mat=:CH3))
        end
        specs = Array{DataSpec}(specs)

        exp_test(ID, specs, "TEST-d=$d")
    end
end

test_unseen_d (generic function with 1 method)

In [16]:
test_unseen_d()

size(mats) = (1600,)
size(mats[1]) = (30, 30)


┌ Info: Loading step-30000.bson ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:38
┌ Info: Generating graphs for 
│   spec = DataSpec(30, 1, :ER, :Gaussian, :CH3, :Linear, 2000, 2, 64, 1234)
└ @ Main /home/hebi/git/DAG-EQ/src/data_graph.jl:568
┌ Info: concating ..
└ @ Main /home/hebi/git/DAG-EQ/src/data_graph.jl:517
┌ Info: done
└ @ Main /home/hebi/git/DAG-EQ/src/data_graph.jl:520


size(mats) = (401,)
size(mats[1]) = (30, 30)


┌ Info: concating ..
└ @ Main /home/hebi/git/DAG-EQ/src/data_graph.jl:517
┌ Info: done
└ @ Main /home/hebi/git/DAG-EQ/src/data_graph.jl:520
┌ Info: generating training ds ..
└ @ Main /home/hebi/git/DAG-EQ/src/data_graph.jl:597
Generating..100%|███████████████████████████████████████| Time: 0:00:01


size(input) = (1600,)
size(input[1]) = (30, 30, 2, 2)
size(output) = (1600,)


...

Inner testing...100%|███████████████████████████████████| Time: 0:00:08
┌ Info: data
│   g_v = GraphMetric: prec: 0.84, recall: 0.86, shd: 11.83
│   loss_v = 32.4844845533371
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:128
┌ Info: Saving results ..
└ @ Main In[4]:41
┌ Info: Loading step-30000.bson ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:38


(xdim, ydim) = (4, 3)
(xdim, ydim) = (4, 3)
ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 100
  x data shape: (30, 30, 2, 6400)
  y data shape: (30, 30, 6400)
  current index: 1]
test_ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 25
  x data shape: (30, 30, 2, 1604)
  y data shape: (30, 30, 1604)
  current index: 1]


Inner testing...100%|███████████████████████████████████| Time: 0:00:01
┌ Info: data
│   g_v = GraphMetric: prec: 0.94, recall: 0.88, shd: 5.01
│   loss_v = 14.129029035568237
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:128
┌ Info: Saving results ..
└ @ Main In[4]:41
┌ Info: Loading step-30000.bson ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:38


(xdim, ydim) = (4, 3)
(xdim, ydim) = (4, 3)
ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 100
  x data shape: (40, 40, 2, 6400)
  y data shape: (40, 40, 6400)
  current index: 1]
test_ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 25
  x data shape: (40, 40, 2, 1604)
  y data shape: (40, 40, 1604)
  current index: 1]


Inner testing...100%|███████████████████████████████████| Time: 0:00:07
┌ Info: data
│   g_v = GraphMetric: prec: 0.93, recall: 0.87, shd: 7.56
│   loss_v = 20.937103867530823
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:128
┌ Info: Saving results ..
└ @ Main In[4]:41


In [22]:
# test the individual trained models
# d=10,15,20
# on larger d
function test_unseen_d_2()
    for ID in ["EQ2-ERSF-k1-d=10-ensemble", "EQ2-ERSF-k1-d=15-ensemble", "EQ2-ERSF-k1-d=20-ensemble"],
        d in [10, 15, 20, 30, 40, 50, 80, 100]

        specs = []

        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                                  noise=:Gaussian, mat=:CH3))
        end
        specs = Array{DataSpec}(specs)

        exp_test(ID, specs, "TEST-d=$d")
    end
end

test_unseen_d_2 (generic function with 1 method)

In [23]:
test_unseen_d_2()

┌ Info: Loading step-30000.bson ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:38


(xdim, ydim) = (4, 3)
(xdim, ydim) = (4, 3)
ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 225
  x data shape: (10, 10, 2, 14400)
  y data shape: (10, 10, 14400)
  current index: 1]
test_ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 56
  x data shape: (10, 10, 2, 3606)
  y data shape: (10, 10, 3606)
  current index: 1]


┌ Warning: calls to Base intrinsics might be GPU incompatible
│   exception = (GPUCompiler.MethodSubstitutionWarning(log1p(x::Float32) in Base.Math at special/log.jl:360, log1p(x::Float32) in CUDA at /home/hebi/.julia/packages/CUDA/YeS8q/src/device/intrinsics/math.jl:85), Base.StackTraces.StackFrame[log1p at log.jl:360, broadcast_kernel at broadcast.jl:59])
└ @ GPUCompiler /home/hebi/.julia/packages/GPUCompiler/uTpNx/src/irgen.jl:68
Inner testing...100%|███████████████████████████████████| Time: 0:00:21
┌ Info: data
│   g_v = GraphMetric: prec: 0.98, recall: 0.93, shd: 0.81
│   loss_v = 2.2192376479506493
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:128
┌ Info: Saving results ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:293
┌ Info: Loading step-30000.bson ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:38


(xdim, ydim) = (4, 3)
(xdim, ydim) = (4, 3)
ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 225
  x data shape: (15, 15, 2, 14400)
  y data shape: (15, 15, 14400)
  current index: 1]
test_ds = DataSetIterator[DataSetIterator:
  batch size: 64
  number of batches: 56
  x data shape: (15, 15, 2, 3606)
  y data shape: (15, 15, 3606)
  current index: 1]


Inner testing...100%|███████████████████████████████████| Time: 0:00:03
┌ Info: data
│   g_v = GraphMetric: prec: 0.81, recall: 0.92, shd: 4.17
│   loss_v = 18.319161772727966
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:128
┌ Info: Saving results ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:293
┌ Info: Loading step-30000.bson ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:38


...

Inner testing...100%|███████████████████████████████████| Time: 0:00:02
┌ Info: data
│   g_v = GraphMetric: prec: 0.57, recall: 0.83, shd: 39.4
│   loss_v = 278.22453117370605
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:128
┌ Info: Saving results ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:293
┌ Info: Loading step-30000.bson ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:38


(xdim, ydim) = (4, 3)
(xdim, ydim) = (4, 3)
ds = DataSetIterator[DataSetIterator:
  batch size: 32
  number of batches: 50
  x data shape: (80, 80, 2, 1600)
  y data shape: (80, 80, 1600)
  current index: 1]
test_ds = DataSetIterator[DataSetIterator:
  batch size: 32
  number of batches: 12
  x data shape: (80, 80, 2, 402)
  y data shape: (80, 80, 402)
  current index: 1]


Inner testing...100%|███████████████████████████████████| Time: 0:00:18
┌ Info: data
│   g_v = GraphMetric: prec: 0.43, recall: 0.72, shd: 97.04
│   loss_v = 769.8894233703613
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:128
┌ Info: Saving results ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:293
┌ Info: Loading step-30000.bson ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:38


(xdim, ydim) = (4, 3)
(xdim, ydim) = (4, 3)
ds = DataSetIterator[DataSetIterator:
  batch size: 16
  number of batches: 100
  x data shape: (100, 100, 2, 1600)
  y data shape: (100, 100, 1600)
  current index: 1]
test_ds = DataSetIterator[DataSetIterator:
  batch size: 16
  number of batches: 25
  x data shape: (100, 100, 2, 402)
  y data shape: (100, 100, 402)
  current index: 1]


Inner testing...100%|███████████████████████████████████| Time: 0:00:12
┌ Info: data
│   g_v = GraphMetric: prec: 0.39, recall: 0.66, shd: 137.13
│   loss_v = 1083.8699226379395
└ @ Main /home/hebi/git/DAG-EQ/src/train.jl:128
┌ Info: Saving results ..
└ @ Main /home/hebi/git/DAG-EQ/src/exp.jl:293


# Test on different noise and the tables

In [ ]:
function test_different_noise()
    # FIXME OOM for d=100??
    for d in [10,20,50,100],
        noise in [:Gaussian, :Poisson, :Exp, :Gumbel]

        ID = "EQ2-ERSF-k1-d=$d-ensemble"

        specs = []
        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                    noise=noise, mat=:CH3))
        end
        specs = Array{DataSpec}(specs)
        
        @info "Testing on" ID d noise
        exp_test(ID, specs, "TEST-d=$d-noise=$noise")
    end
end

In [ ]:
test_different_noise()

# Sachs 2005 experiment

## Load and explore the data

In [ ]:
import CSV

In [ ]:
df = CSV.read("Sachs/1.cd3cd28.csv")

In [ ]:
SachsX = convert(Matrix, df)

In [ ]:
include("data_graph.jl")

In [ ]:
SachsG = Sachs_ground_truth()

In [ ]:
myplot(SachsG)

In [ ]:
medcovX = cov(SachsX) ./ median(var(SachsX, dims=1))

In [ ]:
maxcovX = cov(SachsX) ./ maximum(var(SachsX, dims=1))

In [ ]:
corX = cor(SachsX)

In [ ]:
ch2X = getch2(SachsX)

In [ ]:
include("exp.jl")

## Load a model

In [ ]:
# load the trained model
@load "saved_models/EQ-d=20_k=1_gtype=SF_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/EQ-d=10_k=1_gtype=ER_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "back/back-0907/CORCOV/EQ-d=10_k=1_gtype=SF_noise=Gaussian_mat=COR_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/ensK-2020-09-08T10:58:41.247-ensemble/step-10000.bson" model

In [ ]:
# The new ensemble model
@load "saved_models/ensemEQ-ICLR-1-ensemble/step-159443.bson" model

In [ ]:
@load "saved_models/ensemEQ-CH2-1,2,4-2020-10-11T11:29:01.183-ensemble/step-100000.bson" model

## Evaluate the model

In [ ]:
out = inf_one(model, medcovX)

In [ ]:
out = inf_one(model, corX)

In [ ]:
out = inf_one(model, maxcovX)

In [ ]:
out = inf_one(model, ch2X)

In [ ]:
Wout = threshold(σ.(out), 0.3, true)

In [ ]:
myplot(DiGraph(Wout), names(df))

In [ ]:
# FIXME NOW !!!! the names might not match at all!!!
myplot(SachsG, names(df))

In [ ]:
# predicted edge, true edge, SHD
predicted_edge = ne(DiGraph(Wout))
@show predicted_edge
correct_edge = sum(Wout[Wout .== 1] .== adjacency_matrix(SachsG)[Wout .== 1])
@show correct_edge

# metrics
ytrue = Matrix(gen_weights(SachsG))
sup_graph_metrics(Wout, ytrue)

In [ ]:
# TODO calculate #reverse direction edges
sum(Wout[Wout .== 1] .== adjacency_matrix(SachsG)[Wout .== 1])

In [ ]:
sum(Wout'[Wout' .== 1] .== adjacency_matrix(SachsG)[Wout' .== 1])

In [ ]:
# TODO implement the recursive add procedure to remove cycles
# Or, just construct the graph, and keep removing until it is a DAG
is_cyclic(DiGraph(Wout))

In [ ]:
Wout

In [ ]:
# Or just implement the procedure
out

## Adding non-cyclic procedure

In [ ]:
Wout2 = threshold(σ.(out), 0.3, false)

In [ ]:
# find the order of the index, or, sort the indexes
sort(Wout2, dims=1)

In [ ]:
edgeidx = findall((x)->x>0, Wout2)

In [ ]:
sorted_idx = edgeidx[sortperm(Wout2[edgeidx], rev=true)]

In [ ]:
Wout2

In [ ]:
# add sorted idx
g = MetaDiGraph(11)
for idx in sorted_idx
    add_edge!(g, idx[1], idx[2])
    if is_cyclic(g)
        rem_edge!(g, idx[1], idx[2])
    end
end

In [ ]:
p1 = myplot(g, names(df))

In [ ]:
draw(PNG("p1.png"), p1)

In [ ]:
p0 = myplot(SachsG, names(df))

In [ ]:
draw(PNG("p0.png"), p0)

In [ ]:
adj1 = adjacency_matrix(g)
adj0 = adjacency_matrix(SachsG)
# predicted edge, true edge, SHD
predicted_edge = ne(g)
@show predicted_edge
correct_edge = sum(adj1[adj1 .== 1] .== adj0[adj1 .== 1])
@show correct_edge
reversed_edge = sum(adj1'[adj1' .== 1] .== adj0[adj1' .== 1])
@show reversed_edge

# metrics
ytrue = Matrix(gen_weights(SachsG))
sup_graph_metrics(adj1, ytrue)

## More data

- [ ] goblinop's gaussian data (just for test)
- [ ] bnlearn's continous data
- [ ] discrete data?

### goblinop's (??) gaussian data

In [ ]:
import CSV

In [ ]:
df = CSV.read("data-back/gaussian.dat", delim=" ")

In [ ]:
X = convert(Matrix, df)

In [ ]:
include("data_graph.jl")

In [ ]:
maxcovX = cov(X) ./ maximum(var(X, dims=1))

In [ ]:
include("exp.jl")

In [ ]:
# The new ensemble model
@load "saved_models/ensemEQ-ICLR-1-ensemble/step-159443.bson" model

In [ ]:
out = inf_one(model, maxcovX)

In [ ]:
Wout2 = threshold(σ.(out), 0.3, false)

In [ ]:
# find the order of the index, or, sort the indexes
sort(Wout2, dims=1)

In [ ]:
edgeidx = findall((x)->x>0, Wout2)

In [ ]:
sorted_idx = edgeidx[sortperm(Wout2[edgeidx], rev=true)]

In [ ]:
Wout2

In [ ]:
size(X,2)

In [ ]:
# add sorted idx
g = MetaDiGraph(size(X,2))
for idx in sorted_idx
    add_edge!(g, idx[1], idx[2])
    if is_cyclic(g)
        rem_edge!(g, idx[1], idx[2])
    end
end

In [ ]:
bnp1 = myplot(g, ["A", "B", "C", "D", "E", "F", "G"])

In [ ]:
function bnlearn_ground_truth()
    greal = named_graph([:A, :B, :C, :D, :E, :F, :G])
    named_graph_add_edge!(greal, :B, :C)
    named_graph_add_edge!(greal, :A, :C)
    named_graph_add_edge!(greal, :B, :D)
    named_graph_add_edge!(greal, :D, :F)
    named_graph_add_edge!(greal, :A, :F)
    named_graph_add_edge!(greal, :G, :F)
    named_graph_add_edge!(greal, :E, :F)
    greal
end

In [ ]:
groundG = bnlearn_ground_truth()

In [ ]:
bnp0 = myplot(groundG, ["A", "B", "C", "D", "E", "F", "G"])

In [ ]:
draw(PNG("bnp0.png"), bnp0)

In [ ]:
draw(PNG("bnp1.png"), bnp1)

In [ ]:
adj1 = adjacency_matrix(g)
adj0 = adjacency_matrix(groundG)
# predicted edge, true edge, SHD
predicted_edge = ne(g)
@show predicted_edge
correct_edge = sum(adj1[adj1 .== 1] .== adj0[adj1 .== 1])
@show correct_edge
reversed_edge = sum(adj1'[adj1' .== 1] .== adj0[adj1' .== 1])
@show reversed_edge

# metrics
ytrue = Matrix(gen_weights(groundG))
sup_graph_metrics(adj1, ytrue)